In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# sample list
mapping_16S = pd.read_csv("~/Documents/thesis/16S/qiime2/mappingfiles/16S_mapping.csv")
mapping_16S.head()

,#SampleID,Mother_Child,SubjectID,MaternalID,TimePoint,Mgx_batch Mapping,16S_batch Mapping
0,C0052-5F-1A,C,52,0052_m,5,Mgx_batch001,16S_batch001
1,C0126-8F-1A,C,126,0126_m,8,Mgx_batch001,16S_batch001
2,C0126-8F-1B,C,126,0126_m,8,Mgx_batch001,16S_batch001
3,C0127-6F-1B,C,127,0127_m,6,Mgx_batch001,16S_batch001
4,C0286-5F-1A,C,286,0286_m,5,Mgx_batch001,16S_batch001


In [3]:
samplenames = list(mapping_16S["#SampleID"])

In [4]:
# read in mgx data
mgx = pd.read_csv("mgx_genera.csv")
mgx.rename(columns={'Taxon':'taxa'}, inplace=True)
mgx.head()


,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
0,Acidaminococcaceae_unclassified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
1,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.002632,0.0,0.0,0.000000,0.03501,0.0,0.0
2,Acinetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
3,Actinobacillus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000139,0.000000,0.0,0.0,0.000116,0.00000,0.0,0.0
4,Actinobaculum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [5]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
mgx = mgx[~mgx.taxa.str.contains("_unclassified")]
mgx = mgx[~mgx.taxa.str.contains("_noname")]

In [6]:
mgx.sample(15)

,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
163,Varibaculum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
31,Butyricimonas,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Actinobaculum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
164,Veillonella,0.000419,0.000014,0.000117,0.000000,0.010778,0.000997,0.012216,0.000000,0.066886,...,0.001662,0.053236,0.000401,0.001318,0.000198,0.000000,0.001087,0.000350,0.000736,0.002182
46,Collinsella,0.021758,0.000000,0.000000,0.000000,0.003368,0.032115,0.014475,0.015372,0.000000,...,0.012545,0.000000,0.000000,0.004446,0.000000,0.003000,0.011440,0.004458,0.009387,0.005104
114,Parabacteroides,0.018743,0.004718,0.005318,0.012280,0.053458,0.017058,0.028365,0.034533,0.000000,...,0.044239,0.000000,0.032096,0.032052,0.000000,0.023293,0.025875,0.041084,0.026649,0.010695
57,Dolosigranulum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45,Clostridium,0.000375,0.000000,0.000049,0.013093,0.001150,0.008919,0.000000,0.002350,0.034732,...,0.000232,0.151345,0.002425,0.002827,0.118950,0.000000,0.003554,0.001889,0.005906,0.000000
78,Gluconobacter,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56,Dialister,0.045603,0.000000,0.000000,0.000000,0.001748,0.000000,0.000000,0.016098,0.000000,...,0.008597,0.000000,0.000000,0.000000,0.000000,0.000000,0.026304,0.002382,0.011484,0.000000


In [7]:
mgx.to_csv('mgx_abundance.csv', index=False)

In [8]:
# read in 16S data
amplicon = pd.read_csv("feature-table.csv")
amplicon.rename(columns={'#OTU ID':'taxa'}, inplace=True)


In [9]:
# clean taxonomic names
# keep only genera
amplicon["taxa"] = amplicon['taxa'].str.split("D_5__", expand=True)[1]
amplicon.dropna(inplace=True)


In [10]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
0,Methanobrevibacter,0,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,19,0,0,0
7,Subgroup 10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Subgroup 23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Ilumatobacter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Actinomyces,0,0,7,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
amplicon.sort_values(by = "taxa", ascending=True, inplace=True)

In [12]:
amplicon["taxa"] = amplicon["taxa"].astype(str)

In [13]:
# remove any Nones, uncultured organisms
amplicon = amplicon[~amplicon.taxa.str.contains("uncultured")]
amplicon = amplicon[~amplicon.taxa.str.contains("metagenome")]
amplicon = amplicon[~amplicon.taxa.str.contains("unidentified")]
amplicon = amplicon[~amplicon.taxa.str.contains(r'[0-9]')]
amplicon = amplicon[~amplicon.taxa.str.contains("group")]

In [14]:
# calculate relative abundances
bug_sum = amplicon.sum(axis = 0, skipna = True)
amplicon.iloc[:, 1:] = amplicon.iloc[:, 1:]/bug_sum

In [15]:
amplicon.to_csv('16S_abundance.csv')

### looking for the intersection of taxonomy

In [16]:
# calculate number of taxa in both vs. either one

In [17]:
amplicon_taxa_list = set(amplicon["taxa"])
mgx_taxa_list = set(mgx["taxa"])

In [18]:
intersection = [bug for bug in amplicon_taxa_list if bug in mgx_taxa_list]

def union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

union = union(list(set(amplicon_taxa_list)), list(set(mgx_taxa_list)))


In [19]:
union_df = pd.DataFrame(union)


In [20]:
union_df.head()

,0
0,Megasphaera
1,Butyricicoccus
2,Caldicoprobacter
3,Coprobacter
4,Plesiomonas


In [21]:
union_df.to_csv('taxalist.csv', index=False)

len(intersection)
len(amplicon_taxa_list)-len(intersection)
len(mgx_taxa_list)-len(intersection)

In [22]:
amplicon_only = [bug for bug in amplicon_taxa_list if bug not in intersection]
mgx_only = [bug for bug in mgx_taxa_list if bug not in intersection]


### creating long-form dataframe with data from both sequencing methods

In [23]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
245,Acetanaerobacterium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,Acetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438,Acholeplasma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,Achromobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011094,0.009139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
mgx_unique = [sample for sample in list(mgx.columns.values) if sample not in list(amplicon.columns.values)] 
amplicon_unique = [sample for sample in list(amplicon.columns.values) if sample not in list(mgx.columns.values)]


mgx.drop(mgx_unique, axis=1, inplace=True)
amplicon.drop(amplicon_unique, axis=1, inplace=True)

In [25]:
len(mgx.columns.values)

262

In [26]:
len(amplicon.columns.values)

262

In [27]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [28]:
mgx_melt = pd.melt(mgx, id_vars=["taxa"], var_name = "sampleid", value_name = "mgx_abund")

In [29]:
mgx_melt.head()

,taxa,sampleid,mgx_abund
0,Acidaminococcus,M0652-1F-1A,0.0
1,Acinetobacter,M0652-1F-1A,0.0
2,Actinobacillus,M0652-1F-1A,0.0
3,Actinobaculum,M0652-1F-1A,0.0
4,Actinomyces,M0652-1F-1A,0.0


In [30]:
amplicon_melt.head()

,taxa,sampleid,amplicon_abund
0,Acetanaerobacterium,C0047-7E-1A,0.0
1,Acetobacter,C0047-7E-1A,0.0
2,Acholeplasma,C0047-7E-1A,0.0
3,Achromobacter,C0047-7E-1A,0.0
4,Acidaminococcus,C0047-7E-1A,0.0


In [31]:
merged_taxa = pd.merge(amplicon_melt, mgx_melt, on = ["sampleid", "taxa"], how = "outer")

In [32]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [33]:
merged_taxa.sample(10)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
20011,Escherichia-Shigella,C0839-4F-1A,0.112118,NaN,NaN,NaN
53922,Staphylococcus,M1008-1F-1A,0.000029,0.0,0.000029,0.000029
32179,Fusicatenibacter,M0713-1E-1A,0.002186,NaN,NaN,NaN
71471,Gardnerella,M0890-3F-1A,NaN,0.0,NaN,NaN
17927,Plesiomonas,C0788-2F-1A,0.000000,0.0,0.000000,0.000000
29653,Actinobacillus,M0699-3F-1A,0.000000,0.0,0.000000,0.000000
46598,Planktomarina,M0918-3F-1A,0.000000,NaN,NaN,NaN
9671,Psychroserpens,C0661-2F-1A,0.000000,NaN,NaN,NaN
35784,Cutibacterium,M0744-3F-1A,0.000000,NaN,NaN,NaN
20200,Aurantivirga,C0851-4F-1A,0.000000,NaN,NaN,NaN


In [34]:
merged_taxa.to_csv('taxa_abundance_comparison.csv')

In [35]:
len(mgx_only)
len(amplicon_only)

59

159

In [36]:
# number of samples where there is one bug picked up by 16S and not mgx

rare_16s = merged_taxa[(merged_taxa.amplicon_abund > 0.0) & (merged_taxa.mgx_abund == 0.0)]
samples_rare_16s = (set(rare_16s["sampleid"]))
num_16s_rare_samples = len(samples_rare_16s)

rare_mgx = merged_taxa[(merged_taxa.mgx_abund > 0.0) & (merged_taxa.amplicon_abund == 0.0)]
samples_rare_mgx = (set(rare_mgx["sampleid"]))
num_mgx_rare_samples = len(samples_rare_mgx)


In [37]:
display(num_16s_rare_samples)
display(num_mgx_rare_samples)

259

258

In [38]:
rare_mgx_taxa = rare_mgx["taxa"]
rare_16s_taxa = rare_16s["taxa"]

In [39]:
len(rare_mgx_taxa)
len(rare_16s_taxa)

923

1456

In [40]:
merged_taxa.head(15)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
0,Acetanaerobacterium,C0047-7E-1A,0.000000,NaN,NaN,NaN
1,Acetobacter,C0047-7E-1A,0.000000,NaN,NaN,NaN
2,Acholeplasma,C0047-7E-1A,0.000000,NaN,NaN,NaN
3,Achromobacter,C0047-7E-1A,0.000000,NaN,NaN,NaN
4,Acidaminococcus,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
5,Acinetobacter,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
6,Actibacter,C0047-7E-1A,0.000000,NaN,NaN,NaN
7,Actinobacillus,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
8,Actinomyces,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
9,Adlercreutzia,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000


In [41]:
amplicon_avg_abund = merged_taxa.groupby("taxa")["amplicon_abund"].mean()
mgx_avg_abund = merged_taxa.groupby("taxa")["mgx_abund"].mean()
taxa_list = sorted(set(merged_taxa["taxa"]))

In [42]:
mean_taxa_abund = pd.DataFrame(
    (zip(taxa_list, amplicon_avg_abund, mgx_avg_abund)),  
    columns = ['taxa','amp_avg_abund', 'mgx_avg_abund'])

In [43]:
mean_taxa_abund["abs_diff"] = abs(mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"])
mean_taxa_abund["total_diff"] = mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"]


In [44]:
mean_taxa_abund.sort_values("abs_diff", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [45]:
mean_taxa_abund.sample(10)

,taxa,amp_avg_abund,mgx_avg_abund,abs_diff,total_diff
47,Butyricicoccus,4.124303e-03,0.000014,0.004110,0.004110
230,Polyomavirus,NaN,0.000000,NaN,NaN
284,Thiocystis,5.264744e-07,NaN,NaN,NaN
131,Gemella,9.259277e-05,0.000009,0.000083,0.000083
26,Anaeroglobus,NaN,0.000000,NaN,NaN
80,Desulfatiglans,1.762048e-05,NaN,NaN,NaN
238,Pseudomonas,5.819723e-04,0.000019,0.000563,0.000563
244,Robinsoniella,3.159556e-06,NaN,NaN,NaN
162,Lactonifactor,6.507547e-06,NaN,NaN,NaN
31,Anaerovibrio,1.962000e-05,NaN,NaN,NaN


In [46]:
mean_taxa_abund.to_csv('taxa_difference.csv')

In [47]:
# making giant dataframe of abundances

In [48]:
amp_trans = amplicon.set_index("taxa").transpose()

In [49]:
amp_trans.reset_index(level=0, inplace=True)

In [50]:
amp_trans.rename(columns = {'index':'sampleid'}, inplace = True) 

In [51]:
amp_trans["uid"] = amp_trans["sampleid"].astype(str)+'-amp'

In [52]:
amp_trans["method"] = "amp"

In [53]:
amp_trans.head()

taxa,sampleid,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,Actibacter,Actinobacillus,Actinomyces,...,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts,uid,method
0,C0047-7E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0047-7E-1A-amp,amp
1,C0052-7E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0052-7E-1A-amp,amp
2,C0053-8E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000888,...,0.000634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0053-8E-1A-amp,amp
3,C0058-4E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.003929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0058-4E-1A-amp,amp
4,C0059-4E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.001789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0059-4E-1A-amp,amp


In [54]:
mgx_trans = mgx.set_index("taxa").transpose()

In [55]:
mgx_trans.reset_index(level=0, inplace=True)

In [56]:
mgx_trans.rename(columns = {'index':'sampleid'}, inplace = True)

In [57]:
mgx_trans["uid"] = mgx_trans["sampleid"].astype(str)+'-mgx'

In [58]:
mgx_trans["method"] = "mgx"

In [59]:
mgx_trans.head()

taxa,sampleid,Acidaminococcus,Acinetobacter,Actinobacillus,Actinobaculum,Actinomyces,Adlercreutzia,Aeromonas,Aggregatibacter,Akkermansia,...,Tannerella,Treponema,Tunalikevirus,Turicibacter,Ureaplasma,Varibaculum,Veillonella,Weissella,uid,method
0,M0652-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.001095,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0652-1F-1A-mgx,mgx
1,M0647-1F-1A,0.0,0.0,0.0,0.0,0.0,0.001504,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.001681,0.0,0.0,0.001841,0.0,M0647-1F-1A-mgx,mgx
2,M0656-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000907,0.0,0.0,0.019614,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0656-1F-1A-mgx,mgx
3,C0660-1E-1A,0.0,0.0,0.0,0.0,0.0,0.000575,0.0,0.0,0.005417,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000314,0.0,C0660-1E-1A-mgx,mgx
4,M0649-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0649-1F-1A-mgx,mgx


In [60]:
concat_df = pd.concat([mgx_trans,amp_trans], sort=True).reset_index(drop = True)

In [61]:
age = pd.read_csv("~/Documents/thesis/analysis/metadatawide.csv")

/Users/danielle/miniconda2/envs/thesis_analysis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22,23,99,110,152,212,252,291,294,321,326,415,440,540,547,574,577) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
age = age[['sample','childAgeMonths']]

In [63]:
age.head()

,sample,childAgeMonths
0,C0052_5F_1A,143.0
1,C0126_8F_1A,104.0
2,C0126_8F_1B,104.0
3,C0127_6F_1B,141.0
4,C0286_5F_1A,91.0


In [64]:
age["sample"] = age["sample"].str.replace("_",'-')
age.rename(columns = {'sample':'sampleid'}, inplace = True) 

In [65]:
# make age dictionary
agedict = {str(s): {} for s in age["sampleid"]}
for index, row in age.iterrows():
    age_months = row["childAgeMonths"]
    agedict[row["sampleid"]]= age_months

In [66]:
concat_df["AgeMonths"]= concat_df["sampleid"].map(agedict)

In [72]:
# remove mothers
concat_df = concat_df[~concat_df.sampleid.str.contains("M")]

In [73]:
concat_df.sample(15)

,uid,sampleid,method,AgeMonths,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,...,Ureaplasma,Varibaculum,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts
330,C0768-1E-1A-amp,C0768-1E-1A,amp,52.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,NaN,NaN,0.001225,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0
28,C0123-3E-1A-mgx,C0123-3E-1A,mgx,113.0,NaN,NaN,NaN,NaN,0.0,0.000000,...,0.0,0.0,0.000348,NaN,NaN,NaN,0.0,NaN,NaN,NaN
300,C0661-2F-1A-amp,C0661-2F-1A,amp,34.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,NaN,NaN,0.000369,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0
127,C0925-1F-1A-mgx,C0925-1F-1A,mgx,43.0,NaN,NaN,NaN,NaN,0.0,0.000000,...,0.0,0.0,0.000339,NaN,NaN,NaN,0.0,NaN,NaN,NaN
309,C0692-4F-1A-amp,C0692-4F-1A,amp,16.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,NaN,NaN,0.000259,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0
338,C0811-1E-1A-amp,C0811-1E-1A,amp,57.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,NaN,NaN,0.000000,0.001487,0.0,0.003185,0.0,0.00092,0.0,0.0
323,C0742-3F-1A-amp,C0742-3F-1A,amp,16.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0
197,C0507-2F-1A-mgx,C0507-2F-1A,mgx,110.0,NaN,NaN,NaN,NaN,0.0,0.000000,...,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN
356,C2014-2E-1A-amp,C2014-2E-1A,amp,25.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,NaN,NaN,0.027284,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0
134,C0654-2F-1A-mgx,C0654-2F-1A,mgx,31.0,NaN,NaN,NaN,NaN,0.0,0.000000,...,0.0,0.0,0.005967,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [74]:
cols_to_order = ['uid', 'sampleid',"method", "AgeMonths"]
new_columns = cols_to_order + (concat_df.columns.drop(cols_to_order).tolist())
concat_df = concat_df[new_columns]

In [75]:
concat_df.sample(15)

,uid,sampleid,method,AgeMonths,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,...,Ureaplasma,Varibaculum,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts
252,C0828-4F-1A-mgx,C0828-4F-1A,mgx,13.0,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.0,0.0,0.053236,NaN,NaN,NaN,0.0,NaN,NaN,NaN
173,C1034-1F-1A-mgx,C1034-1F-1A,mgx,24.0,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.0,0.0,0.003795,NaN,NaN,NaN,0.0,NaN,NaN,NaN
301,C0672-1E-1A-amp,C0672-1E-1A,amp,39.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.002734,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
354,C1102-1F-1A-amp,C1102-1F-1A,amp,92.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
322,C0739-2F-1A-amp,C0739-2F-1A,amp,17.0,0.0,0.0,0.0,0.0,0.000126,0.006064,...,NaN,NaN,0.151071,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
162,C0713-4F-1A-mgx,C0713-4F-1A,mgx,16.0,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.0,0.0,0.015230,NaN,NaN,NaN,0.0,NaN,NaN,NaN
330,C0768-1E-1A-amp,C0768-1E-1A,amp,52.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.001225,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
280,C0402-3F-1A-amp,C0402-3F-1A,amp,77.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.000105,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
229,C0839-4F-1A-mgx,C0839-4F-1A,mgx,12.0,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.0,0.0,0.019127,NaN,NaN,NaN,0.0,NaN,NaN,NaN
125,C0754-2F-1A-mgx,C0754-2F-1A,mgx,18.0,NaN,NaN,NaN,NaN,0.000000,0.000019,...,0.0,0.0,0.039616,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [76]:
concat_df.to_csv('transposed_mgxamp_df.csv')